In [ ]:
# check GPU status
!nvidia-smi

In [26]:
!pip install -q librosa
!pip install -q opendatasets

In [16]:
"""This is an Ellen issue. For some reason I keep on getting path issue on the modules."""
import sys
print(sys.path)

PATH = "/projectnb/ec500kb/students/eburhan/Project/venvs/mynewenv/lib/python3.10/site-packages"
sys.path.append(PATH)

['/projectnb/dl523/students/eburhan/project_SER', '', '/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.10/site-packages', '/share/pkg.8/python3/3.10.12/install/lib/python310.zip', '/share/pkg.8/python3/3.10.12/install/lib/python3.10', '/share/pkg.8/python3/3.10.12/install/lib/python3.10/lib-dynload', '/usr4/ec500kb/eburhan/.local/lib/python3.10/site-packages', '/share/pkg.8/python3/3.10.12/install/lib/python3.10/site-packages', '/projectnb/ec500kb/students/eburhan/Project/venvs/mynewenv/lib/python3.10/site-packages']


In [23]:
# Import libraries
import math
import random
import IPython
import os
import torch
import torchvision
import librosa
import torchaudio
# import torchvision.transforms as transforms
from torchaudio.transforms import MelSpectrogram
from torchaudio import transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import pandas as pd

from torch.utils.data import Dataset, DataLoader, random_split
from torchaudio.utils import download_asset
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torchvision import datasets
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

SyntaxError: invalid syntax (1940498688.py, line 11)

In [3]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [4]:
from torch.utils.data import DataLoader
from dataset import download_datasets, SpeechEmotionDataset, get_dataset_info

# Specify the directory you want the datasets to be contained in
dataset_dir = './datasets'
os.makedirs(dataset_dir, exist_ok=True)

####
# Only needed to run this code once
####
## Download a single dataset
#download_datasets(dataset_dir, dname="emodb")
#
## Download the rest of the datasets available
#download_datasets(dataset_dir)


# Acquire info on datasets (those that have functions to get data for)
df = get_dataset_info(dataset_dir)

# Make into a Dataset object that a pytorch optimizer can use
# Can optionally specify a sampling rate for all audio files to be in
trainset = SpeechEmotionDataset(df, fs=16000)

# Check it works
dataiter = iter(trainset)
data, label = next(dataiter)
print(data)
print(label)
print(df) # columns are: filename, speaker_n, intensity, emotion, version, language, database 

# # Put into a dataloader
# trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=1)


tensor([ 9.1553e-05, -3.0518e-04, -7.9346e-04,  ..., -1.2207e-03,
        -1.4343e-03, -1.5259e-03])
anxiety
                                                filename speaker_n intensity  \
0      ./datasets/berlin-database-of-emotional-speech...        16        NA   
1      ./datasets/berlin-database-of-emotional-speech...        10        NA   
2      ./datasets/berlin-database-of-emotional-speech...        16        NA   
3      ./datasets/berlin-database-of-emotional-speech...        16        NA   
4      ./datasets/berlin-database-of-emotional-speech...        14        NA   
...                                                  ...       ...       ...   
15692  ./datasets/shemo-persian-speech-emotion-detect...        56        NA   
15693  ./datasets/shemo-persian-speech-emotion-detect...        12        NA   
15694  ./datasets/shemo-persian-speech-emotion-detect...        12        NA   
15695  ./datasets/shemo-persian-speech-emotion-detect...        04        NA   
15696  ./da

In [6]:
### Data
X = df['filename']
print(X, "\n")

### Label encoding features
print("Emotions: ", df['emotion'].unique(), "\n")
print("Num of classes: ", len(df['emotion'].unique()), "\n")


0        ./datasets/berlin-database-of-emotional-speech...
1        ./datasets/berlin-database-of-emotional-speech...
2        ./datasets/berlin-database-of-emotional-speech...
3        ./datasets/berlin-database-of-emotional-speech...
4        ./datasets/berlin-database-of-emotional-speech...
                               ...                        
15692    ./datasets/shemo-persian-speech-emotion-detect...
15693    ./datasets/shemo-persian-speech-emotion-detect...
15694    ./datasets/shemo-persian-speech-emotion-detect...
15695    ./datasets/shemo-persian-speech-emotion-detect...
15696    ./datasets/shemo-persian-speech-emotion-detect...
Name: filename, Length: 15697, dtype: object 

Emotions:  ['anxiety' 'bored' 'neutral' 'disgust' 'anger' 'sadness' 'happy'
 'surprise' 'fear' 'calm'] 

Num of classes:  10 



In [7]:
print("Shape of dataframe: ", df.shape, "\n")

column_headers = df.columns
print("Column headers of dataframe: ", column_headers, "\n")

df_subset = df[['filename', 'emotion']]

# Perform one-hot encoding on 'emotion'
# Integer encoding
label_encoder = LabelEncoder()
integer_encoding = label_encoder.fit_transform(df_subset['emotion'])
print('Integer encoding: ', integer_encoding, "\n")

# Binary encoding
one_hot_encoder = OneHotEncoder(sparse=False)
integer_encoding = integer_encoding.reshape(-1, 1)
one_hot_encoding = one_hot_encoder.fit_transform(integer_encoding)
print('One hot encoding: ', one_hot_encoding, "\n")

# One-hot encoding to DataFrame
one_hot_df = pd.DataFrame(one_hot_encoding, columns=label_encoder.classes_)
result_df = pd.concat([df_subset['filename'], one_hot_df], axis=1)

# Combining emotions into one array for each file name (drop individual one-hot encoded columns)
result_df['emotion_onehot'] = result_df.iloc[:, 1:].values.tolist()
result_df.drop(result_df.columns[1:-1], axis=1, inplace=True)

print("Current dataframe: ")
print(result_df)


Shape of dataframe:  (15697, 7) 

Column headers of dataframe:  Index(['filename', 'speaker_n', 'intensity', 'emotion', 'version', 'language',
       'database'],
      dtype='object') 

Integer encoding:  [1 2 2 ... 0 7 0] 

One hot encoding:  [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]] 

Current dataframe: 
                                                filename  \
0      ./datasets/berlin-database-of-emotional-speech...   
1      ./datasets/berlin-database-of-emotional-speech...   
2      ./datasets/berlin-database-of-emotional-speech...   
3      ./datasets/berlin-database-of-emotional-speech...   
4      ./datasets/berlin-database-of-emotional-speech...   
...                                                  ...   
15692  ./datasets/shemo-persian-speech-emotion-detect...   
15693  ./datasets/shemo-persian-speech-emotion-detect...   
15694  ./datasets/shemo-persian-speec

/share/pkg.8/python3/3.10.12/install/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
"""
Adapted from: https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5
"""
class audio_preprocessing():
    def read_file(file):
        signal, sample_rate = torchaudio.load(file)
        
        return (signal, sample_rate)
    
    # ----------------------------
    # Standardize number of audio channels
    # ---------------------------
    def set_num_channel(audio, desired_num_channel):
        signal, sample_rate = audio
        
        if(signal.shape[0] == desired_num_channel): # No change
            return audio
        
        if(desired_num_channel == 1): # Converting stereo to mono
            new_signal = signal[:1, :]
        else:
            new_signal = torch.cat([signal, signal])
            
        return ((new_signal, sample_rate))
    
    # ----------------------------
    # Standardize sampling rate
    # ---------------------------    
    def set_sampling_rate(audio, new_sr):
        signal, sampling_rate = audio
        
        if(sampling_rate == new_sr):
            return audio
        
        num_channels = signal.shape[0]
        
        # Resampling first channel
        channel_1 = torchaudio.transforms.Resample(sampling_rate, new_sr)(signal[:1,:])
        
        if (num_channels > 1):
            # Resample the second channel and merge both channels
            channel_2 = torchaudio.transforms.Resample(sampling_rate, new_sr)(signal[1:,:])
            resample = torch.cat([channel_1, channel_2])
        else:
            resample = channel_1

        return ((resample, new_sr))
    
    
    # ----------------------------
    # Standardize length of audio samples
    # max_ms = milliseconds
    # --------------------------- 
    def standardize_audio_length(audio, max_ms):
        signal, sampling_rate = audio
        num_rows, signal_len = signal.shape
        max_len = sampling_rate//1000 * max_ms

        if (signal_len > max_len):
          # Truncate the signal to the given length
          signal = signal[:,:max_len]

        elif (signal_len < max_len):
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - signal_len)
            pad_end_len = max_len - signal_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            signal = torch.cat((pad_begin, signal, pad_end), 1)
      
        return (signal, sampling_rate)

    # ----------------------------
    # Shifts the signal to the left or right by some percent. Values at the end
    # are 'wrapped around' to the start of the transformed signal.
    # ----------------------------
    def time_shift(audio, shift_limit): # Not sure if we need this
        signal, sample_rate = audio
        _, signal_len = signal.shape
        shift_amt = int(random.random() * shift_limit * signal_len)
        
        return (signal.roll(shift_amt), sample_rate)
    
    # ----------------------------
    # Generate a Spectrogram
    # ----------------------------
    def generate_mfcc_spectrogram(audio, n_mels=64, n_fft=1024, hop_len=None):
        signal,sample_rate = audio
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = torchaudio.transforms.MelSpectrogram(sample_rate, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(signal)

        # Convert to decibels
        spec = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
        
        return (spec)
    
    # ----------------------------
    # Augment the Spectrogram by masking out some sections of it in both the frequency
    # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
    # overfitting and to help the model generalise better. The masked sections are
    # replaced with the mean value.
    # ----------------------------
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        
        for _ in range(n_time_masks):
            aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec


In [25]:
"""
Creating Data Loader
"""
# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
    def __init__(self, df):
        self.df = df
        self.duration = 4000
        self.sr = 16000
        self.channel = 1
        self.shift_pct = 0.4
            
    # ----------------------------
    # Number of items in dataset
    # ----------------------------
    def __len__(self):
        return len(self.df)    

    # ----------------------------
    # Get i'th item in dataset
    # ----------------------------
    def __getitem__(self, idx):
        # Extracting filename and one-hot encoded emotions
        filename = self.df.loc[idx, 'filename']
        emotion_onehot = torch.tensor(self.df.loc[idx, 'emotion_onehot'], dtype=torch.float32)

        audio = audio_preprocessing.read_file(filename)
        # Some sounds have a higher sample rate, or fewer channels compared to the
        # majority. So make all sounds have the same number of channels and same 
        # sample rate. Unless the sample rate is the same, the pad_trunc will still
        # result in arrays of different lengths, even though the sound duration is
        # the same.
        reaud = audio_preprocessing.set_sampling_rate(audio, self.sr)
        rechan = audio_preprocessing.set_num_channel(reaud, self.channel)

        dur_aud = audio_preprocessing.standardize_audio_length(rechan, self.duration)
        shift_aud = audio_preprocessing.time_shift(dur_aud, self.shift_pct)
        sgram = audio_preprocessing.generate_mfcc_spectrogram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        aug_sgram = audio_preprocessing.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

        return aug_sgram, emotion_onehot

In [29]:
# result_df consists of filename and the one-hot encoded emotions
dataset = SoundDS(result_df)

# Random split with ratios of 70% training, 10% validation, and 20% testing
total_items = len(dataset)
train_size = round(total_items * 0.7)
val_size = round(total_items * 0.1)
test_size = total_items - train_size - val_size

# Checking dataset split
print("Size of dataset: ", total_items)
print("Size of Training data (%): ", train_size / total_items * 100)
print("Size of Testing data (%): ", test_size / total_items * 100)
print("Size of Validation data (%): ", val_size / total_items * 100)

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

# Create training and validation data loaders
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False)


Size of dataset:  15697
Size of Training data (%):  70.00063706440721
Size of Testing data (%):  19.997451742371155
Size of Validation data (%):  10.001911193221636


In [32]:
# Check size of mel spectrogram at the end: 
# e.g. (num_channels, Mel freq_bands, time_steps in spec) = (2, 64, 344)
sample_data, _ = next(iter(train_loader))
# print(sample_data)

# # Assuming sample_data is a tuple containing the Mel Spectrogram and emotion one-hot encoding
mel_spectrogram = sample_data[0] # Assuming Mel Spectrogram is at index 0


mel_shape = mel_spectrogram.shape
print("Size of Mel Spectrogram:", mel_shape, "\n")



Size of Mel Spectrogram: torch.Size([1, 64, 126]) 



In [45]:
# '''
# Feature Extraction
# 1) Wav2vec2 model
# '''
# bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H # can choose another wav2vec2 model
# print("Sample rate: ", bundle.sample_rate)
# print("Labels: ", bundle.get_labels())
# model = bundle.get_model().to(device)

# # Example audio
# SPEECH_FILE = download_asset("tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav")
# IPython.display.Audio(SPEECH_FILE)
# waveform, sample_rate = torchaudio.load(SPEECH_FILE)
# waveform = waveform.to(device)
# # Resample example audio if its sample rate doesn't match the pipeline's sample rate
# if sample_rate != bundle.sample_rate:
#     print("Audio vs Bundle sample rate: ", sample_rate, bundle.sample_rate)
#     waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
    
# # Extract features
# with torch.inference_mode():
#     features, _ = model.extract_features(waveform)
    

In [8]:
# '''
# Feature Extraction
# 2) Hand-crafted features: ZCR, RMSE, MFCC
# '''
# X, sample_rate = librosa.load('./datasets/berlin-database-of-emotional-speech-emodb/wav/03a01Fa.wav')
# mfccs = np.mean(librosa.feature.mfcc(y=X, n_mfcc=25,), axis = 0) # calculating mean?
# rms = librosa.feature.rms(y=X) # root mean square value for each frame of audio sample
# zcr = librosa.feature.zero_crossing_rate(y=X) # zero crossing rate of audio time series

In [44]:
# # Let's visualize what we are doing!
# # some of this code taken from https://github.com/MiteshPuthran/Speech-Emotion-Analyzer/blob/master/final_results_gender_test.ipynb
# import librosa.display
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10,5))
# librosa.display.waveshow(X, sr=sample_rate)

In [10]:
# putting features into dataframes
# needs "/" at end of filepath
data_filepath = "./datasets/berlin-database-of-emotional-speech-emodb/wav/"

database_name = "berlin-database-of-emotional-speech-emodb"
filenames_list = os.listdir(data_filepath) # filenames without full filepath
full_filenames_list = [data_filepath + filename for filename in filenames_list] # adding full filepath



In [43]:
# # extract features of all audio files into a dataframe
# import pandas as pd

# features_df = pd.DataFrame(columns=['filename', 'mfccs', 'rms', 'zcr', 'emotion'])

# # works for a single folder of audio files at once, all from the same dataset
# for i, filename in enumerate(full_filenames_list):
#     X, sample_rate = librosa.load(filename) # load waveform
#     mfccs = np.mean(librosa.feature.mfcc(y=X, n_mfcc=25,), axis = 0) # calculate feature values
#     rms = librosa.feature.rms(y=X)
#     zcr = librosa.feature.zero_crossing_rate(y=X)
#     features_df.at[i, 'mfccs'] = mfccs # save X features to dataframe
#     features_df.at[i, 'rms'] = np.array(rms)
#     features_df.at[i, 'zcr'] = np.array(zcr)
    
#     row_index = df.loc[df['filename'] == filename].index[0] # finding the correct emotion for the filename
#     features_df.at[i, 'emotion'] = df.at[row_index,'emotion'] # selects correct emotion from Noah's df
    
#     split_name = filename.split('/') # get the correct filename
#     features_df.at[i, 'filename'] = split_name[-1]
    
#     #features_df.at[i,'emotion'] = df.loc['im','emotion']

# #features_df = pd.concat([features_df, labels_df], axis=1)
# print(features_df)

In [12]:
# Now let's shuffle these boiz
from sklearn.model_selection import train_test_split
train, test = train_test_split(features_df, test_size=0.2, random_state=42, shuffle=True)


train shape: (428, 5)



In [22]:
"""
Building Network architecture based on the paper:
Human–Computer Interaction with a Real-Time Speech
Emotion Recognition with Ensembling Techniques 1D
Convolution Neural Network and Attention
(https://doi.org/10.3390/s23031386)

We are taking the output of CNN as the input of LSTM.
CNN captures local patterns in audio features, and
LSTM learns temporal dependencies before making final prediction. This supports
robust sequence prediction.
"""
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        # bn = batch normalization
        ####################
        # Convolution blocks: 1dconv, batch norm, ReLU, max pooling
        # Conv block 1
        self.conv1 = nn.Conv1d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1)
        self.bn1 = nn.BatchNorm1d(8)

        # Conv block 2
        self.conv2 = nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1)
        self.bn2 = nn.BatchNorm1d(16)

        # Conv block 3
        self.conv3 = nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, padding = 1)
        self.bn3 = nn.BatchNorm1d(32)

        # Conv block 4
        self.conv4 = nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1)
        self.bn4 = nn.BatchNorm1d(64)

        # Conv block 5
        self.conv5 = nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
        self.bn5 = nn.BatchNorm1d(128)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size = 2, stride = 2)
        ####################

        ####################
        # LSTM + attention block
        self.lstm1 = nn.LSTM(input_size = 128, hidden_size = 64, num_layers = 1, bias = True)

        self.attention = nn.Linear(64, 1)
        self.softmax = nn.Softmax(dim=1)

        self.lstm2 = nn.LSTM(input_size = 64, hidden_size = 64, num_layers = 1, bias = True)
        ####################


        self.fc1 = nn.Linear(64, 32) # May need to increase 32 to capture more complex data (?)
        self.bn6 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 16)


    def forward(self, x):
        # Conv block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)

        # Conv block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)

        # Conv block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)

        # Conv block 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pool(x)

        # Conv block 5
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu(x)
        x = self.pool(x)

        # LSTM + attention block
        # output_tensor, hiddenstate = self.lstm()
        lstm1_out, _ = self.lstm1(x)

        attention_weights = self.softmax(self.attention(lstm1_out))
        context = torch.sum(attention_weights * lstm1_out, dim=1)

        lstm2_out, _ = self.lstm2(context.unsqueeze(-1))

        # fully connected layers and softmax
        x = self.fc1(lstm2_out.squeeze(1))
        x = self.bn6(x)
        x = self.fc2(x)

        # Softmax for prediction
        # F.softmax is used instead of self.softmax because it is not associated
        # with any parameters
        x = F.softmax(x, dim=1)

        return x

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net =  CNN_LSTM().to(device)
criterion = nn.CrossEntropyLoss()
optimizer =  optim.Adam(net.parameters(), lr=1e-4)

In [36]:
# Training loop
root_dir = './runs'
os.makedirs(root_dir, exist_ok=True)

def train_on_features(net, optimizer, device, trainloader, critrerion, epochs=1):
    if torch.cuda.is_available():
        net.cuda()
    net.train()

    loss_min = float('inf')
    for epoch in range(epochs):
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = net(inputs)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            
            if loss < loss_min:
                torch.save(net.state_dict(), os.path.join(root_dir, "best_ser_model.pth"))


    print('Finished Training')
    return net

net = train_on_features(net, optimizer, device, trainloader, criterion)

AttributeError: 'tuple' object has no attribute 'to'

In [37]:
#net = train_on_features(net, optimizer, device, trainloader, criterion)
# trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=1)
# for i, data in enumerate(trainloader):
#     print(data)
print(trainset[0])

(tensor([ 9.1553e-05, -3.0518e-04, -7.9346e-04,  ..., -1.2207e-03,
        -1.4343e-03, -1.5259e-03]), 'anxiety')
